# Natural-language processing_2-layer LSTM classifier
Project: Build A Emojifier for Sentiment Analysis                                           --By Junde Li on 27 Feb 2018


By using word vectors, even if training set explicitly relates only a few words to a particular emoji, the algorithm will be able to generalize and associate words in the test set to the same emoji even if those words don't even appear in the training set. This model will enable accurate classifier mapping from sentences to emojis with small data examples.

In [1]:
import numpy as np
import csv
import pandas as panda
import emoji
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

Using TensorFlow backend.


# 1.1 Data Loading

Load the dataset using csv, and split the dataset between training and testing sets.

In [2]:
# Read in dataset by csv.
phrase = []
emoji = []
with open ('data_emoji.csv') as csvfile:
    items = csv.reader(csvfile)
    
    for i in items:
        phrase.append(i[0])
        emoji.append(i[1])
        
    X = np.asarray(phrase)
    y = np.asarray(emoji, dtype=int)

In [3]:
# Randomly split 20% of training set as test set.
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [4]:
n_train = X_train.shape[0]
n_test = X_test.shape[0]

# Unique classes/labels there in the dataset.
n_classes = len(np.unique(Y_train))

maxLen = len(max(X_train, key=len).split())

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Number of classes =", n_classes)
print("Number of maximum lenth of sentences =", maxLen)

Number of training examples = 191
Number of testing examples = 48
Number of classes = 5
Number of maximum lenth of sentences = 10


# 1.2 Data Visulization

Run the following cell to print 5 different labels from Y_train and corresponding sentences from X_train. Because of the font the iPython notebook uses, the heart emoji may be colored black rather than red.

In [5]:
import emoji
# Converts a label (int or string) into the corresponding emoji code (string)
def label_to_emoji(label):
    # Set up our own emoji dictionary for output (Here I just set up five emojis).
    emoji_dictionary = {"0": "\u2764\uFE0F",
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)

classes = []
for i in range(len(X_train)):
    index = np.random.randint(len(X_train))
    if Y_train[index] not in classes:
        print(X_train[index], label_to_emoji(Y_train[index]))
        classes.append(Y_train[index])
    if Y_train[index] in classes:
        continue
    # Break the for-loop if all classes show up.
    if len(classes) == n_classes:
        break

Are you down for baseball this afternoon ⚾
will you be my valentine	 ❤️
cookies are good 🍴
he got a raise 😄
I am upset	 😞


# 2. Design and test Emojifier: Using LSTMs in Keras: 

Below builds an LSTM model that takes as input word sequences. This model will be able to take word ordering into account. Emojifier will continue to use pre-trained word embeddings to represent words, but will feed them into an LSTM, whose job it is to predict the most appropriate emoji.

Note: In order to train Keras using mini-batches, padding will be used to set all sentences to the same maximum lenth.

# 2.1 The Embedding Layer

Because the training set is quite small, we will not update the word embeddings but will instead leave their values fixed.

The Embedding() layer takes an integer matrix of size (batch size, max input length) as input. This corresponds to sentences converted into lists of indices.

In [6]:
def read_glove_vecs(glove_file):
    with open(glove_file, encoding='utf-8', mode = 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [7]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [8]:
def sentences_to_indices(X, word_to_index, max_len):
    
    m = X.shape[0]
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):
        
        # Convert the ith training sentence in lower case and split is into words.
        sentence_words = X[i].lower().split()
        j = 0
        
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j = j+1
            
    return X_indices

In [9]:
# Pretrained_embedding layer by importing the glove.6B.50d.txt (downloaded from https://www.kaggle.com/rohitanil/glove6b50dtxt).
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    # Add 1 to fit Keras embedding (requirement).
    vocab_len = len(word_to_index) + 1              
    emb_dim = word_to_vec_map["good"].shape[0]
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation.
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, input_length=None, trainable = False)
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

# 2.2 Build the Emojifier

In [10]:
def Emojify(input_shape, word_to_vec_map, word_to_index):
    
    # Define sentence_indices as the input of the graph.
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through embedding layer.
    embeddings = embedding_layer(sentence_indices)
    
    # An LSTM layer with 128-dimensional hidden state (128 controls the memory lenth) with output of a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    # X = LSTM(128, return_sequences=True)(X)   # 2-layer LSTM give good enough result.
    # X = Dropout(0.5)(X)
    X = LSTM(128)(X)
    X = Dropout(0.5)(X)
    
    # A Dense layer to get back a batch of 5-dimensional vectors.
    X = Dense(5)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

In [11]:
model = Emojify((maxLen,), word_to_vec_map, word_to_index)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
__________

# 2.3 Train and test Emojifier

Before training the model, I convert the sentences and labels to indices and one-hot vectors separately for feeding.

In [12]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = np.eye(5)[Y_train.reshape(-1)]

In [13]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
191/191 [==============================] - 1s 6ms/step - loss: 1.5734 - acc: 0.2618
Epoch 2/50
191/191 [==============================] - 0s 871us/step - loss: 1.4809 - acc: 0.3194
Epoch 3/50
191/191 [==============================] - 0s 835us/step - loss: 1.4614 - acc: 0.3665
Epoch 4/50
191/191 [==============================] - 0s 871us/step - loss: 1.3427 - acc: 0.4555
Epoch 5/50
191/191 [==============================] - 0s 903us/step - loss: 1.2178 - acc: 0.5236
Epoch 6/50
191/191 [==============================] - 0s 772us/step - loss: 1.0684 - acc: 0.6126
Epoch 7/50
191/191 [==============================] - 0s 990us/step - loss: 0.8735 - acc: 0.6963
Epoch 8/50
191/191 [==============================] - 0s 934us/step - loss: 0.7929 - acc: 0.7539
Epoch 9/50
191/191 [==============================] - 0s 866us/step - loss: 0.5849 - acc: 0.8063
Epoch 10/50
191/191 [==============================] - 0s 856us/step - loss: 0.4843 - acc: 0.8482
Epoch 11/50
191/191 [==========

In [14]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = np.eye(5)[Y_test.reshape(-1)]
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)
print("The accuracy is achieved by only 191 training samples.")

48/48 [==============================] - 0s 2ms/step

Test accuracy =  0.833333333333
The accuracy is achieved by only 191 training samples.


In [15]:
# Below show mislabelled examples.
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😄 prediction: What you did was awesome❤️
Expected emoji:😞 prediction: you got a down grade❤️
Expected emoji:😞 prediction: you suck😄
Expected emoji:😄 prediction: What you did was awesome	❤️
Expected emoji:⚾ prediction: lets exercise😄
Expected emoji:😞 prediction: This girl is messing with me❤️
Expected emoji:😞 prediction: the party is cancelled😄
Expected emoji:😞 prediction: I am very disappointed😄


Now try our own examples. Please make sure all the words we write are in the Glove embeddings.

In [16]:
x_test = np.array(['Thanks for your reading'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

Thanks for your reading 😄


# Acknowledgments

Thanks to Dr. Andrew Ng and his team for providing the learning materials and insights for this project.